# 🏎️ F1 Visual Difference Engine - Interactive Demo

**Multi-Pipeline Visual Change Detection System**

This notebook demonstrates the complete F1 Visual Difference Engine with all 4 pipelines:
- **Semantic Pipelines**: DINO, CLIP
- **Anomaly Pipelines**: PatchCore, PaDiM

Run all cells to process the sample image pairs and compare pipeline performance.

## 1. Setup and Import Libraries

In [ ]:
import sys
import os

# Add parent directory to path for imports
sys.path.append(os.path.abspath('..'))

# Standard libraries
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

# Import our custom modules
from main_pipeline import run_all_pipelines

# Configure matplotlib
%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 10)
plt.rcParams['figure.dpi'] = 100

print("✅ All imports successful!")
print(f"Python version: {sys.version}")
print(f"OpenCV version: {cv2.__version__}")
print(f"NumPy version: {np.__version__}")

## 2. Load and Display Sample Image Pairs

We have 4 test cases in the `samples/` folder:
1. **Livery Change**: back1.jpeg ↔ back2.jpeg
2. **Object Change**: copy1.jpeg ↔ copy2.jpeg  
3. **Tire Damage**: crack1.jpg ↔ crack2.png
4. **Subtle Change**: side1.jpeg ↔ side2.jpeg

In [ ]:
# Define sample image pairs
sample_pairs = {
    "Livery Change": ("../samples/back1.jpeg", "../samples/back2.jpeg"),
    "Object Change": ("../samples/copy1.jpeg", "../samples/copy2.jpeg"),
    "Tire Damage": ("../samples/crack1.jpg", "../samples/crack2.png"),
    "Subtle Change": ("../samples/side1.jpeg", "../samples/side2.jpeg")
}

# Display all image pairs
fig, axes = plt.subplots(4, 2, figsize=(12, 16))
fig.suptitle("Sample Image Pairs", fontsize=16, fontweight='bold')

for idx, (name, (ref_path, test_path)) in enumerate(sample_pairs.items()):
    # Load images
    ref_img = cv2.imread(ref_path)
    test_img = cv2.imread(test_path)
    
    # Convert BGR to RGB for display
    ref_img = cv2.cvtColor(ref_img, cv2.COLOR_BGR2RGB)
    test_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB)
    
    # Display
    axes[idx, 0].imshow(ref_img)
    axes[idx, 0].set_title(f"{name} - Reference", fontweight='bold')
    axes[idx, 0].axis('off')
    
    axes[idx, 1].imshow(test_img)
    axes[idx, 1].set_title(f"{name} - Test", fontweight='bold')
    axes[idx, 1].axis('off')

plt.tight_layout()
plt.show()

print("✅ All sample image pairs loaded successfully!")

## 3. Run Complete Pipeline on Selected Pair

Select which image pair to analyze in detail. The pipeline will:
1. Preprocess (resize, denoise, gamma correct, SIFT align)
2. Generate rough mask (SSIM)
3. Refine with SAM
4. Compute routing features
5. Run ALL 4 pipelines
6. Generate LLaVA reports

In [ ]:
# Select which pair to analyze (change this to test different pairs)
selected_pair = "Livery Change"  # Options: "Livery Change", "Object Change", "Tire Damage", "Subtle Change"

ref_path, test_path = sample_pairs[selected_pair]

print(f"🚀 Running pipeline on: {selected_pair}")
print(f"   Reference: {ref_path}")
print(f"   Test: {test_path}")
print("\n" + "="*60)

# Run complete pipeline
output = run_all_pipelines(ref_path, test_path)

print("\n" + "="*60)
print(f"✅ Pipeline execution complete!")
print(f"\nPredicted Type: {output['predicted_type'].upper()}")
print(f"Confidence: {output['confidence'].upper()}")
print(f"\nRouting Features:")
for key, value in output['features'].items():
    print(f"  {key}: {value:.4f}")

## 4. Preprocessing Results

Visualize the preprocessing steps: aligned images, rough mask, and SAM-refined mask.

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(20, 5))
fig.suptitle("Preprocessing Pipeline Results", fontsize=14, fontweight='bold')

# Aligned reference
axes[0].imshow(output['preprocessed']['ref'])
axes[0].set_title("Aligned Reference", fontweight='bold')
axes[0].axis('off')

# Aligned test
axes[1].imshow(output['preprocessed']['test'])
axes[1].set_title("Aligned Test", fontweight='bold')
axes[1].axis('off')

# Rough mask (SSIM)
rough_rgb = cv2.cvtColor(output['preprocessed']['rough_mask'], cv2.COLOR_GRAY2RGB)
axes[2].imshow(rough_rgb)
axes[2].set_title("Rough Mask (SSIM)", fontweight='bold')
axes[2].axis('off')

# Refined mask (SAM)
refined_rgb = cv2.cvtColor(output['preprocessed']['refined_mask'], cv2.COLOR_GRAY2RGB)
axes[3].imshow(refined_rgb)
axes[3].set_title("Refined Mask (SAM)", fontweight='bold')
axes[3].axis('off')

plt.tight_layout()
plt.show()

## 5. Semantic Pipeline Results: DINO & CLIP

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle("Semantic Pipelines: DINO & CLIP", fontsize=16, fontweight='bold')

# DINO results
axes[0, 0].imshow(output['results']['dino']['heatmap'])
axes[0, 0].set_title("DINO - Heatmap", fontweight='bold')
axes[0, 0].axis('off')

dino_mask_rgb = cv2.cvtColor(output['results']['dino']['mask_final'], cv2.COLOR_GRAY2RGB)
axes[0, 1].imshow(dino_mask_rgb)
axes[0, 1].set_title("DINO - Mask", fontweight='bold')
axes[0, 1].axis('off')

axes[0, 2].imshow(output['results']['dino']['overlay'])
axes[0, 2].set_title("DINO - Overlay", fontweight='bold')
axes[0, 2].axis('off')

# CLIP results
axes[1, 0].imshow(output['results']['clip']['heatmap'])
axes[1, 0].set_title("CLIP - Heatmap", fontweight='bold')
axes[1, 0].axis('off')

clip_mask_rgb = cv2.cvtColor(output['results']['clip']['mask_final'], cv2.COLOR_GRAY2RGB)
axes[1, 1].imshow(clip_mask_rgb)
axes[1, 1].set_title("CLIP - Mask", fontweight='bold')
axes[1, 1].axis('off')

axes[1, 2].imshow(output['results']['clip']['overlay'])
axes[1, 2].set_title("CLIP - Overlay", fontweight='bold')
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

# Display reports
print("\n" + "="*60)
print("📄 DINO REPORT:")
print("="*60)
print(output['results']['dino']['report'])

print("\n" + "="*60)
print("📄 CLIP REPORT:")
print("="*60)
print(output['results']['clip']['report'])

## 6. Anomaly Pipeline Results: PatchCore & PaDiM

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle("Anomaly Pipelines: PatchCore & PaDiM", fontsize=16, fontweight='bold')

# PatchCore results
axes[0, 0].imshow(output['results']['patchcore']['heatmap'])
axes[0, 0].set_title(f"PatchCore - Heatmap (Severity: {output['results']['patchcore']['severity']:.3f})", fontweight='bold')
axes[0, 0].axis('off')

patchcore_mask_rgb = cv2.cvtColor(output['results']['patchcore']['mask_final'], cv2.COLOR_GRAY2RGB)
axes[0, 1].imshow(patchcore_mask_rgb)
axes[0, 1].set_title("PatchCore - Mask", fontweight='bold')
axes[0, 1].axis('off')

axes[0, 2].imshow(output['results']['patchcore']['overlay'])
axes[0, 2].set_title("PatchCore - Overlay", fontweight='bold')
axes[0, 2].axis('off')

# PaDiM results
axes[1, 0].imshow(output['results']['padim']['heatmap'])
axes[1, 0].set_title(f"PaDiM - Heatmap (Severity: {output['results']['padim']['severity']:.3f})", fontweight='bold')
axes[1, 0].axis('off')

padim_mask_rgb = cv2.cvtColor(output['results']['padim']['mask_final'], cv2.COLOR_GRAY2RGB)
axes[1, 1].imshow(padim_mask_rgb)
axes[1, 1].set_title("PaDiM - Mask", fontweight='bold')
axes[1, 1].axis('off')

axes[1, 2].imshow(output['results']['padim']['overlay'])
axes[1, 2].set_title("PaDiM - Overlay", fontweight='bold')
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

# Display reports
print("\n" + "="*60)
print("📄 PATCHCORE REPORT:")
print("="*60)
print(output['results']['patchcore']['report'])

print("\n" + "="*60)
print("📄 PADIM REPORT:")
print("="*60)
print(output['results']['padim']['report'])

## 7. Complete 4-Pipeline Comparison Grid

Side-by-side comparison of all pipelines for easy visual assessment.

In [ ]:
fig = plt.figure(figsize=(20, 16))
gs = GridSpec(4, 3, figure=fig)
fig.suptitle(f"Complete Pipeline Comparison: {selected_pair}", fontsize=18, fontweight='bold')

pipeline_names = ["DINO (Semantic)", "CLIP (Semantic)", "PatchCore (Anomaly)", "PaDiM (Anomaly)"]
pipeline_keys = ["dino", "clip", "patchcore", "padim"]

for idx, (name, key) in enumerate(zip(pipeline_names, pipeline_keys)):
    result = output['results'][key]
    
    # Heatmap
    ax1 = fig.add_subplot(gs[idx, 0])
    ax1.imshow(result['heatmap'])
    ax1.set_title(f"{name}\nHeatmap", fontweight='bold', fontsize=11)
    ax1.axis('off')
    
    # Mask
    ax2 = fig.add_subplot(gs[idx, 1])
    mask_rgb = cv2.cvtColor(result['mask_final'], cv2.COLOR_GRAY2RGB)
    ax2.imshow(mask_rgb)
    ax2.set_title("Segmentation Mask", fontweight='bold', fontsize=11)
    ax2.axis('off')
    
    # Overlay
    ax3 = fig.add_subplot(gs[idx, 2])
    ax3.imshow(result['overlay'])
    title = "Overlay"
    if 'severity' in result:
        title += f" (Severity: {result['severity']:.3f})"
    ax3.set_title(title, fontweight='bold', fontsize=11)
    ax3.axis('off')

plt.tight_layout()
plt.show()

print("\n" + "="*80)
print("📊 PIPELINE COMPARISON SUMMARY")
print("="*80)
print(f"Image Pair: {selected_pair}")
print(f"Predicted Type: {output['predicted_type'].upper()} (Confidence: {output['confidence']})")
print(f"\nRouting Features:")
for key, value in output['features'].items():
    print(f"  • {key}: {value:.4f}")
print("\n" + "="*80)

## 8. Test All Image Pairs (Batch Processing)

Run the complete pipeline on all 4 sample pairs to demonstrate generalization and robustness.

In [ ]:
# Store all results
all_results = {}

print("🚀 Running pipeline on all 4 image pairs...\n")

for name, (ref_path, test_path) in sample_pairs.items():
    print(f"Processing: {name}")
    print("-" * 60)
    
    result = run_all_pipelines(ref_path, test_path)
    all_results[name] = result
    
    print(f"  ✅ Type: {result['predicted_type'].upper()} (Confidence: {result['confidence']})")
    print(f"  Features: texture_var={result['features']['texture_var']:.1f}, "
          f"edge_density={result['features']['edge_density']:.3f}, "
          f"color_shift={result['features']['color_shift']:.3f}")
    print()

print("\n" + "="*80)
print("✅ ALL IMAGE PAIRS PROCESSED SUCCESSFULLY")
print("="*80)

## 9. Summary Dashboard: All Pairs & Routing Predictions

In [ ]:
# Create summary table
print("\n" + "="*100)
print("📊 COMPREHENSIVE RESULTS SUMMARY")
print("="*100)
print(f"\n{'Image Pair':<20} {'Type':<12} {'Conf':<8} {'Texture':<10} {'Edge':<10} {'Color':<10}")
print("-" * 100)

for name, result in all_results.items():
    print(f"{name:<20} {result['predicted_type'].upper():<12} "
          f"{result['confidence']:<8} "
          f"{result['features']['texture_var']:<10.1f} "
          f"{result['features']['edge_density']:<10.3f} "
          f"{result['features']['color_shift']:<10.3f}")

print("="*100)

# Expected vs Predicted
print("\n" + "="*100)
print("🎯 EXPECTED vs PREDICTED PIPELINE PERFORMANCE")
print("="*100)

expectations = {
    "Livery Change": {"type": "semantic", "best": ["DINO", "CLIP"]},
    "Object Change": {"type": "mixed", "best": ["CLIP", "PatchCore"]},
    "Tire Damage": {"type": "anomaly", "best": ["PatchCore", "PaDiM"]},
    "Subtle Change": {"type": "semantic", "best": ["DINO"]}
}

for name, expect in expectations.items():
    result = all_results[name]
    print(f"\n{name}:")
    print(f"  Expected Type: {expect['type']}")
    print(f"  Predicted Type: {result['predicted_type']} ({result['confidence']} confidence)")
    print(f"  Expected Best Pipelines: {', '.join(expect['best'])}")
    print(f"  ✅ Match!" if expect['type'] in result['predicted_type'] or expect['type'] == "mixed" else "  ⚠️ Different prediction")

print("\n" + "="*100)

## 10. Conclusion & Manual Pipeline Selection

Based on the results above, you can manually assess which 2 pipelines performed best for each task.

### Expected Performance:

| Task | Expected Type | Best Pipelines | Reason |
|------|---------------|----------------|--------|
| Livery Change | Semantic | DINO, CLIP | Color/design changes |
| Object Change | Mixed | CLIP, PatchCore | Semantic + structural |
| Tire Damage | Anomaly | PatchCore, PaDiM | Texture irregularities |
| Subtle Change | Semantic | DINO | Fine-grained features |

### Next Steps:
1. Review visualizations for each pipeline
2. Compare heatmap quality and mask precision
3. Read LLaVA reports for semantic understanding
4. Select top 2 pipelines based on your task requirements

---

**🏁 Demo Complete!** The F1 Visual Difference Engine successfully demonstrates:
- ✅ Multi-pipeline architecture (4 concurrent approaches)
- ✅ Intelligent routing (semantic vs anomaly prediction)
- ✅ SAM-based refinement for high-quality masks
- ✅ LLaVA-powered natural language reports
- ✅ Generalization across different change types